### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [78]:
!pip install langchain

In [79]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Path.cwd() in a notebook is the folder containing the notebook
env_path = Path.cwd() / ".env"

if env_path.exists():
    load_dotenv(dotenv_path=env_path, override=True)
    print(f"✅ Successfully loaded .env from: {env_path}")
else:
    print(f"❌ Could not find .env at: {env_path}")

# VERIFICATION: This MUST print True before you run the model cell
print("GROQ_API_KEY loaded:", bool(os.getenv("GROQ_API_KEY")))


❌ Could not find .env at: D:\Projects\Gen\14. Langchain\LCEL\.env
GROQ_API_KEY loaded: True


In [80]:
!pip install langchain_groq

In [81]:
import os
from langchain_groq import ChatGroq

# Ensure load_dotenv() was called before this
model = ChatGroq(
    model="llama-3.1-8b-instant",
    groq_api_key=os.getenv("GROQ_API_KEY")
)

In [82]:
import os

# 1. Get the string from the environment
my_key = os.getenv("GROQ_API_KEY")

# 2. Pass that string to the model
model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=my_key)

In [83]:
!pip install langchain_core

In [84]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello, How are you?")
]

result=model.invoke(messages)

In [85]:
result

AIMessage(content='Bonjour, comment ça va ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 48, 'total_tokens': 55, 'completion_time': 0.017703257, 'completion_tokens_details': None, 'prompt_time': 0.002295408, 'prompt_tokens_details': None, 'queue_time': 0.055434872, 'total_time': 0.019998665}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019befa6-7df3-7f52-9354-5e514aaee6ee-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 48, 'output_tokens': 7, 'total_tokens': 55})

In [86]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment ça va ?'

In [87]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, Comment allez-vous ?'

In [88]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [89]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [90]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [91]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour'

In [92]:
!pip install streamlit